In [35]:
import requests
import sqlite3
from lxml import html
from bs4 import BeautifulSoup
from datetime import date,datetime
import pandas as pd 
import numpy as np

In [48]:
# Read items data

items_df = pd.read_csv("items.csv")
items_df = items_df.drop(["link", "title", "image", "category1_id", "category1_name", "brand", "price", "likes", "description", "keyword1", "keyword2", "keyword3", "keyword4", "keyword5", "keyword6"], axis=1)
items_df = items_df.sort_values('item_id').reset_index(drop=True)
items_df['category1'] = ""
items_df['category2'] = ""
items_df['category3'] = ""

items_df

,item_id,category2_id,category2_name,category3_id,category3_name,category4_id,category4_name,category1,category2,category3
0,10517,60,Jewelry,67,Brooches,0,NaN,,,
1,12045,41,Shoes,46,Sandals,0,NaN,,,
2,14024,318,Handbags,38,Clutches,38,Swarovski clutches,,,
3,36366,11,Tops,19,Sweaters,19,DAY Birger et Mikkelsen sweaters,,,
4,74666,60,Jewelry,64,Earrings,0,NaN,,,
5,78059,2,Clothing,0,NaN,0,NaN,,,
6,98959,11,Tops,104,Tank Tops,104,American Eagle Outfitters tops,,,
7,99084,60,Jewelry,65,Rings,0,NaN,,,
8,105737,2,Clothing,23,Outerwear,24,Coats,,,
9,106678,60,Jewelry,62,Necklaces,0,NaN,,,


In [19]:
len(items_df)

355169

In [65]:
import json

# Read item category json data
with open('item_category.json' , 'r') as reader:
    category_json = json.loads(reader.read())

category1_list = []
category2_list = []
category3_list = []
category1_dic = {}
category2_dic = {}
category3_dic = {}
category1_map_category2 = {}
category2_map_category1 = {}
category2_map_category3 = {}
category3_map_category2 = {}
    
# Print to see the contents
for i in xrange(len(category_json)):

    print "Category1-"+str(i)+" "+category_json[i]['category1_name']+"(id="+str(category_json[i]['category1_id'])+")"
    category1_list.append(category_json[i]['category1_id'])
    category1_dic[category_json[i]['category1_id']] = category_json[i]['category1_name']
    category1_map_category2[category_json[i]['category1_id']] = []
    
    for j in xrange(len(category_json[i]['sub_categories'])):
        
        print "    Category2-"+str(j)+" "+category_json[i]['sub_categories'][j]['category2_name']+"(id="+str(category_json[i]['sub_categories'][j]['category2_id'])+")"
        category2_list.append(category_json[i]['sub_categories'][j]['category2_id'])
        category2_dic[category_json[i]['sub_categories'][j]['category2_id']] = category_json[i]['sub_categories'][j]['category2_name']
        category1_map_category2[category_json[i]['category1_id']].append(category_json[i]['sub_categories'][j]['category2_id'])
        category2_map_category1[category_json[i]['sub_categories'][j]['category2_id']] = category_json[i]['category1_id']
        category2_map_category3[category_json[i]['sub_categories'][j]['category2_id']] = []
        
        for k in xrange(len(category_json[i]['sub_categories'][j]['sub_categories'])):
        
            print "        Category3-"+str(k)+" "+category_json[i]['sub_categories'][j]['sub_categories'][k]['category3_name']+"(id="+str(category_json[i]['sub_categories'][j]['sub_categories'][k]['category3_id'])+")"
            category3_list.append(category_json[i]['sub_categories'][j]['sub_categories'][k]['category3_id'])
            category3_dic[category_json[i]['sub_categories'][j]['sub_categories'][k]['category3_id']] = category_json[i]['sub_categories'][j]['sub_categories'][k]['category3_name']
            category2_map_category3[category_json[i]['sub_categories'][j]['category2_id']].append(category_json[i]['sub_categories'][j]['sub_categories'][k]['category3_id'])
            category3_map_category2[category_json[i]['sub_categories'][j]['sub_categories'][k]['category3_id']] = category_json[i]['sub_categories'][j]['category2_id']
            
    print "\n"

# Print category mapping list
print "Category1"
print category1_list
print category1_dic
print "\n"
print "Category1 map Category2"
print category1_map_category2
print "Category2 map Category1"
print category2_map_category1
print "\n"
print "Category2"
print category2_list
print category2_dic
print "\n"
print "Category2 map Category3"
print category2_map_category3
print "Category3 map Category2"
print category3_map_category2
print "\n"
print "Category3"
print category3_list
print category3_dic


Category1-0 Clothing(id=2)
    Category2-0 Dresses(id=3)
        Category3-0 Day(id=4)
        Category3-1 Cocktail(id=5)
        Category3-2 Gowns(id=6)
    Category2-1 Skirts(id=7)
        Category3-0 Mini(id=8)
        Category3-1 Knee Length(id=9)
        Category3-2 Long(id=10)
    Category2-2 Tops(id=11)
        Category3-0 Blouses(id=17)
        Category3-1 Cardigans(id=18)
        Category3-2 Sweaters(id=19)
        Category3-3 Sweatshirts & Hoodies(id=20)
        Category3-4 Tanks(id=104)
        Category3-5 T-Shirts(id=21)
        Category3-6 Tunics(id=15)
    Category2-3 Outerwear(id=23)
        Category3-0 Coats(id=24)
        Category3-1 Jackets(id=25)
        Category3-2 Vests(id=26)
    Category2-4 Jeans(id=27)
        Category3-0 Bootcut(id=238)
        Category3-1 Boyfriend(id=240)
        Category3-2 Flared(id=4452)
        Category3-3 Skinny(id=237)
        Category3-4 Straight Leg(id=310)
        Category3-5 Wide Leg(id=239)
    Category2-5 Pants(id=28)
        Cate

In [80]:
for i in xrange(len(items_df)):
    
    if items_df.category2_id[i] in category1_list:
        items_df.category1[i] = items_df.category2_id[i]
        
    elif items_df.category2_id[i] in category2_list:
        items_df.category2[i] = items_df.category2_id[i]
        items_df.category1[i] = category2_map_category1[items_df.category2[i]]
        
    elif items_df.category2_id[i] in category3_list:
        items_df.category3[i] = items_df.category2_id[i]
        items_df.category2[i] = category3_map_category2[items_df.category3[i]]
        items_df.category1[i] = category2_map_category1[items_df.category2[i]]
        
    if items_df.category3_id[i] in category2_list:
        items_df.category2[i] = items_df.category3_id[i]
        
    elif items_df.category3_id[i] in category3_list:
        items_df.category3[i] = items_df.category3_id[i]
        items_df.category2[i] = category3_map_category2[items_df.category3[i]]
        

/Users/michellechen/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/michellechen/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/michellechen/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/michellechen/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingW

In [81]:
items_df

,item_id,category2_id,category2_name,category3_id,category3_name,category4_id,category4_name,category1,category2,category3
0,10517,60,Jewelry,67,Brooches,0,NaN,51,60,67
1,12045,41,Shoes,46,Sandals,0,NaN,41,46,
2,14024,318,Handbags,38,Clutches,38,Swarovski clutches,51,35,318
3,36366,11,Tops,19,Sweaters,19,DAY Birger et Mikkelsen sweaters,2,11,19
4,74666,60,Jewelry,64,Earrings,0,NaN,51,60,64
5,78059,2,Clothing,0,NaN,0,NaN,2,,
6,98959,11,Tops,104,Tank Tops,104,American Eagle Outfitters tops,2,11,104
7,99084,60,Jewelry,65,Rings,0,NaN,51,60,65
8,105737,2,Clothing,23,Outerwear,24,Coats,2,23,
9,106678,60,Jewelry,62,Necklaces,0,NaN,51,60,62


In [82]:
import csv

items_df = items_df.drop(["category2_id", "category2_name", "category3_id", "category3_name", "category4_id", "category4_name"], axis=1)
items_df

# Write new csv to store reset category data
items_df.to_csv('item_category.csv', encoding='utf-8', index=False)

In [ ]:
# Generate item category into json format

import json

data = []

category1_num = 3
category1_id = [2, 41, 51]
category1_name = ['Clothing', 'Shoes', 'Accessories']
category1_link = ['https://www.polyvore.com/clothing/shop?category_id=2', 
                  'https://www.polyvore.com/shoes/shop?category_id=41', 
                  'https://www.polyvore.com/accessories/shop?category_id=51']
# Headers
head = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.65 Safari/537.36'}

for i in xrange(category1_num):
    res = requests.get(category1_link[i], headers = head)
    soup = BeautifulSoup(res.text, "html.parser")
    
    # Initialize category2 
    category2_num = 0
    category2_id = []
    category2_name = []
    category2_link = []
    sub_categories1 = []
    
    if i == 0:
        secondary = soup.select('.tertiary_nav.new_masthead.clearfix')[0].select('.tab')[0:-1]
    else:
        secondary = soup.select('.tertiary_nav.new_masthead.clearfix')[0].select('.tab')
        
    # line in secondary categories
    for line in secondary:
        category2_link.append('https://www.polyvore.com/' + line['href'][3:])
        category2_id.append(int(line['href'].split('id=')[-1]))
        category2_name.append(line.text)
        
        # Initialize category3
        category3_num = 0
        category3_id = []
        category3_name = []
        category3_link = []
        sub_categories2 = []
        
        for category2 in soup.select('.mod_secondary_subnav'):
            
            # If category2 has sub categories
            if category2.select('a')[0].text == category2_name[category2_num]: 
        
                # line in third categories
                for sub_line in category2.select('.dock_bl')[0].select('li'):
                    category3_link.append('https://www.polyvore.com/' + sub_line.select('a')[0]['href'][3:])
                    category3_id.append(int(sub_line.select('a')[0]['href'].split('id=')[-1]))
                    category3_name.append(sub_line.select('a')[0].text)
                    
                    sub_categories2.append({'category3_id': category3_id[category3_num],
                                            'category3_name': category3_name[category3_num],
                                            'category3_link': category3_link[category3_num]})
                    
                    category3_num = category3_num + 1
        
        sub_categories1.append({'category2_id': category2_id[category2_num],
                                'category2_name': category2_name[category2_num],
                                'category2_link': category2_link[category2_num],
                                'sub_categories': sub_categories2})
        
        category2_num = category2_num + 1
        
    data.append({'category1_id': category1_id[i], 
                 'category1_name': category1_name[i], 
                 'category1_link': category1_link[i],
                 'sub_categories': sub_categories1})

json = json.dumps(data , sort_keys=True, indent=4, separators=(',', ': '), encoding='utf-8')
print json

# Write json data into the file
with open('item_category.json', 'w') as f:
    f.write(json)
    